In [1]:
from src.features.extract_features import *
from src.data.dataset import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = r"F:\01UOttawa\Graduation project\models notebook\Dataset\Semi-final data"

In [6]:
from sklearn.metrics import accuracy_score, recall_score, balanced_accuracy_score, f1_score

def eval_clf(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average = 'macro')
    fscore = f1_score(y_true, y_pred, average = 'macro')
    return acc, balanced_acc, recall, fscore

In [7]:
from lightgbm import LGBMClassifier as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss

In [19]:
model = lgb(learning_rate = 0.1, n_estimators = 2000)#boosting_type = 'rf', learning_rate = 0.05, n_estimators = 2000, bagging_freq = 20, bagging_fraction = 0.5)

feature_sets = [#['RMS'],
                #['ssc', 'zc', 'WL'],
                ['mav', 'ssc', 'zc', 'WL'], # Hudgins
                ['RMS', 'ssc', 'zc', 'WL'],
                ['Var', 'ssc', 'zc', 'WL'],
                ['mav', 'RMS', 'ssc', 'zc', 'WL'],
                ['mav', 'Var', 'ssc', 'zc', 'WL'],
                ['mav', 'RMS', 'Var', 'ssc', 'zc', 'WL'],
                ['iEMG', 'WAMP', 'Var', 'ssc', 'zc', 'WL']]
                #['skewness', 'RMS', 'iEMG', 'hjorth_params', ''],
                #['mmnf', 'mmnf', 'mdf', 'mmdf']]

In [20]:
for fs in feature_sets:
    dataset = EmgDataset(data_dir, win_size = 200, win_stride = 40, feature_set = fs)
    emg_features = dataset.extracted_features
    labels = dataset.rolled_labels
    reps = dataset.rolled_repetition
    train_rows = np.isin(reps, ['1','3','4']).ravel()
    test_rows = np.isin(reps, ['2']).ravel()

    train_emg = emg_features[train_rows]
    y_train = labels[train_rows].ravel()

    test_emg = emg_features[test_rows]
    y_test = labels[test_rows].ravel()
    
    #under sampling
#     rus = RandomUnderSampler(random_state = 0, sampling_strategy = 'majority')
#     x_res_rus, y_res_rus = rus.fit_resample(train_emg, y_train)
    
#     undersample = NearMiss(version=1, n_neighbors=3)
#     x_res_rus, y_res_rus = undersample.fit_resample(train_emg, y_train)
    
    
    model.fit(train_emg, y_train)
    train_preds = model.predict(train_emg)
    test_preds = model.predict(test_emg)
    train_acc, bal_train_acc, train_recall, train_fscore = eval_clf(y_train, train_preds)
    test_acc, bal_test_acc, test_recall, test_fscore = eval_clf(y_test, test_preds)
    
    print(f'Features: {fs}')
    print(f"train acc: {train_acc} test acc: {test_acc}")
    print(f"balanced train acc: {bal_train_acc} balanced test acc: {bal_test_acc}")
    print(f"train recall: {train_recall} test recall: {test_recall}")
    print(f"train f1_score: {train_fscore} test f1_score: {test_fscore}")
    print('==================================================================================')
    

Features: ['mav', 'ssc', 'zc', 'WL']
train acc: 1.0 test acc: 0.7962232671014713
balanced train acc: 1.0 balanced test acc: 0.7015868142839049
train recall: 1.0 test recall: 0.7015868142839049
train f1_score: 1.0 test f1_score: 0.7007455111174341
Features: ['RMS', 'ssc', 'zc', 'WL']
train acc: 1.0 test acc: 0.8016330451488953
balanced train acc: 1.0 balanced test acc: 0.7129196961105585
train recall: 1.0 test recall: 0.7129196961105585
train f1_score: 1.0 test f1_score: 0.7108599499057336
Features: ['Var', 'ssc', 'zc', 'WL']
train acc: 1.0 test acc: 0.8114667071136054
balanced train acc: 1.0 balanced test acc: 0.7279051460152761
train recall: 1.0 test recall: 0.7279051460152761
train f1_score: 1.0 test f1_score: 0.7261499472026056
Features: ['mav', 'RMS', 'ssc', 'zc', 'WL']
train acc: 1.0 test acc: 0.8041356994792457
balanced train acc: 1.0 balanced test acc: 0.7148727152380793
train recall: 1.0 test recall: 0.7148727152380793
train f1_score: 1.0 test f1_score: 0.7151506669086456
Featu

In [ ]:
from collections import Counter

print('y_train')
print(Counter(y_train).keys())
print(Counter(y_train).values())
print('==================================================================================')

print('y under sampling')
print(Counter(y_res_rus).keys())
print(Counter(y_res_rus).values())
print('==================================================================================')

print('y_test')
print(Counter(y_test).keys())
print(Counter(y_test).values())
print('==================================================================================')

print('test_preds')
print(Counter(test_preds).keys())
print(Counter(test_preds).values())
print('==================================================================================')
